<a href="https://colab.research.google.com/github/cpython-projects/da_2603/blob/main/lesson_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sqlalchemy import create_engine, text
import plotly.express as px

In [2]:
DB_USER = ""
DB_PASS = ""
DB_HOST = ""
DB_PORT = ""
DB_NAME = ""

In [3]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

У нас есть пользователи, которые:

* устанавливают приложение (`app_sessions`),
* смотрят товары (`product_views_log`),
* авторизуются (`devices_users_map`),
* совершают покупки (`orders_log`).

# Работа с датами в SQL
**Основные функции для работы с датой для анализа данных**

In [ ]:
query = "SELECT * FROM app_sessions LIMIT 5;"
df = pd.read_sql(text(query), engine)
df

,session_id,device_code,first_seen,os_type,acquisition_channel,cpi_uah
0,S00000,DVC0000,2024-04-25,Android,Facebook,30.53
1,S00001,DVC0001,2024-01-15,Android,Referral,13.91
2,S00002,DVC0002,2024-01-18,Android,Facebook,31.80
3,S00003,DVC0003,2024-02-23,iOS,Referral,14.98
4,S00004,DVC0004,2024-04-24,Android,Google Ads,29.64


## `DATE_TRUNC`  
**обрезает дату или время до заданной единицы измерения**, обнуляя менее значимые компоненты.

### Общий синтаксис:

```sql
DATE_TRUNC('единица_времени', timestamp)
```

---

### Для чего применяем:

1. **Группировка по времени**:
   Чтобы сгруппировать данные по неделям, месяцам, дням и т.п.
2. **Округление до начала часа, дня и т.п.**
3. **Сравнение дат на уровне нужной точности** (например, все события за конкретный месяц).

---

### Поддерживаемые единицы:

* `millennium`, `century`, `decade`, `year`, `quarter`, `month`
* `week`, `day`, `hour`, `minute`, `second`

## `EXTRACT`
**выделение отдельной компоненты даты или времени**, например: год, месяц, день, час и т.д.

### Синтаксис:

```sql
EXTRACT(единица_времени FROM дата_или_время)
```

---

### Используемые единицы:

* `YEAR`
* `MONTH`
* `DAY`
* `HOUR`
* `MINUTE`
* `SECOND`
* `DOW` (day of week: 0 = Sunday, 6 = Saturday)
* `DOY` (day of year)
* `WEEK`
* `QUARTER`

---

### Для чего применяем:

1. **Фильтрация по части даты**:
2. **Создание временных группировок**:

---

### Разница с `DATE_TRUNC`:

|                | `EXTRACT`                     | `DATE_TRUNC`                                 |
| -------------- | ----------------------------- | -------------------------------------------- |
| Что делает     | Возвращает одно число         | Обнуляет менее значимые части даты           |
| Тип результата | `numeric`                     | `timestamp`                                  |
| Пример         | `EXTRACT(MONTH FROM d)` → `7` | `DATE_TRUNC('month', d)` → `'2025-07-01...'` |

## `CURRENT_DATE`, `CURRENT_TIMESTAMP` и `NOW()`
**текущее время и/или дата**

### Особенности

| Функция             | Что возвращает                 | Тип данных  | Пример результата            |
| ------------------- | ------------------------------ | ----------- | ---------------------------- |
| `CURRENT_DATE`      | Только текущую дату            | `DATE`      | `2025-07-14`                 |
| `CURRENT_TIMESTAMP` | Текущие дата **и** время       | `TIMESTAMP` | `2025-07-14 11:25:32.123456` |
| `NOW()`             | То же, что `CURRENT_TIMESTAMP` | `TIMESTAMP` | `2025-07-14 11:25:32.123456` |

## `INTERVAL`  
 **Прибавление/вычитание временного промежутка**: дни, часы, месяцы, секунды и т.д.

### 📌 Синтаксис:

```sql
SELECT CURRENT_DATE + INTERVAL '7 days';
```
---

## Единицы измерения:

* `second`, `minute`, `hour`
* `day`, `week`, `month`, `year`
* И даже комбинации:

```sql
INTERVAL '2 days 3 hours 15 minutes'
```
---

## Особенности:

* `INTERVAL` сам по себе не дата — это просто **временной отрезок**.
* Может использоваться и с `DATE`, и с `TIMESTAMP`.
* В некоторых СУБД, например MySQL, `INTERVAL` пишется иначе:

```sql
-- MySQL стиль
SELECT NOW() + INTERVAL 1 DAY;
```
---

### В Pandas аналог:

```python
import pandas as pd

pd.Timestamp.now() + pd.Timedelta(days=3, hours=2)
```

## Когда применяем работу с датами и временем

Анализ по дате и времени нужен в ситуациях, когда мы хотим:

| Цель анализа                               | Когда применять                                               | Что нужно                         |
| ------------------------------------------ | ------------------------------------------------------------- | --------------------------------- |
| **1. Графики по дням / неделям / месяцам** | Отслеживать динамику показателей                              | Группировка по дате               |
| **2. Сезонность**                          | Найти циклы в поведении (например, пятничные пики покупок)    | Группировка по дню недели, месяцу |
| **3. Повторяемость покупок**               | Сколько дней проходит между покупками                         | Расчёт интервалов между датами    |
| **4. RFM-анализ**                          | Сегментировать пользователей по их активности и ценности      | DATEDIFF, COUNT, SUM              |
| **5. Retention**                           | Понимать, возвращаются ли пользователи после первого действия | Когорты по дате первого события   |

### 1. Графики **по дням, неделям, месяцам**

**Зачем**: смотреть динамику продаж, посещений, заказов во времени.

**Как**: используем `DATE_TRUNC`, `GROUP BY`, агрегаты.

In [ ]:
import plotly.express as px

In [12]:
# Сколько установок в день



In [13]:
# Сколько заказов по неделям


In [14]:
# Сколько просмотров в месяц


### 2. **Cезонность и циклы** (ежемесячная/еженедельная активность)

**Зачем**: находить пики и спады по времени года, неделе, дню недели.

**Как**: `EXTRACT(MONTH FROM date)`, `EXTRACT(DOW FROM date)`.

In [15]:
# Сколько заказов по дням недели



### 3. Повторяемость покупок

**Зачем**: определить, как часто пользователи возвращаются и сколько времени проходит между покупками.

**Как**: сравниваем `DATEDIFF` между заказами.

In [16]:
# Сколько дней между первой и последней покупкой


In [17]:
# Связь между количеством заказов и "жизненным циклом" пользователя


In [18]:
# сколько пользователей возвращаются через X дней?


### 4. Анализ RFM (Recency, Frequency, Monetary)

**Зачем**: **метод сегментации клиентов** по их поведению в отношении заказов, который помогает понять ценность клиента и его лояльность.


#### Что означает каждая буква?

| Параметр      | Что измеряет                    | Пример расчета                                            |
| ------------- | ------------------------------- | --------------------------------------------------------- |
| **Recency**   | Как давно был последний заказ   | Разница между сегодняшней датой и датой последнего заказа |
| **Frequency** | Как часто клиент делает покупки | Количество заказов за период                              |
| **Monetary**  | Сколько денег клиент принес     | Сумма всех заказов                                        |

---

#### Для чего используется RFM?

* **Сегментация**: найти VIP-клиентов, "спящих" клиентов, новых и т.д.
* **Персонализация**: настроить email-маркетинг или push-уведомления
* **Предсказание оттока**: кто может уйти

**Задача — оценить активность клиентов**:
* Как давно был последний заказ (**Recency**),
* Сколько всего заказов сделал клиент (**Frequency**),
* На какую сумму клиент потратился (**Monetary**).


In [19]:
# Запрос для расчета RFM-показателей


In [20]:
# Разделение по квантилям (R/F/M-баллы)


**Recency (давность последней покупки)**

```python
rfm["R"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1]).astype(int)
```

* `pd.qcut()` делит `recency` на **5 квантилей** (примерно равные по размеру группы).
* `labels=[5, 4, 3, 2, 1]` — меньшее значение `recency` (то есть недавняя покупка) получает **лучший балл 5**.
* То есть: покупал вчера → `recency = 1` → `R = 5`

*Почему обратный порядок (5, 4, ..., 1)?*

Потому что чем **меньше дней прошло с последней покупки**, тем **ценнее клиент**.

---

**Frequency (частота покупок)**

```python
rfm["F"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5]).astype(int)
```

* `rank(method="first")` присваивает уникальные ранги (если значения одинаковые).
* Затем `pd.qcut(..., 5)` разбивает их на 5 равных групп.
* `labels=[1, 2, 3, 4, 5]` — чаще покупает → выше `frequency` → выше балл `F`

---

**Monetary (сколько потратил)**

```python
rfm["M"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5]).astype(int)
```

* Всё просто: чем больше тратит, тем выше балл.
* `qcut` делит пользователей по объёму трат на 5 групп от 1 (мало) до 5 (много).

---

**Сборка итогового RFM-скора**

```python
rfm["RFM_score"] = rfm["R"].astype(str) + rfm["F"].astype(str) + rfm["M"].astype(str)
```

* Склеивает значения `R`, `F` и `M` в строку, например:
  `R=5`, `F=3`, `M=4` → `RFM_score = "534"`

---

**В результате:**  

| user\_id | R | F | M | RFM\_score |
| -------- | - | - | - | ---------- |
| 101      | 5 | 3 | 4 | "534"      |
| 102      | 2 | 5 | 5 | "255"      |

---

In [21]:
# Интерпретация сегментов


In [22]:
# Визуализация: Plotly scatter


### 5. **Retention (удержание пользователей)**


#### Зачем?

**Retention** показывает, возвращаются ли пользователи после первого взаимодействия с продуктом (например, после установки приложения).

> Мы хотим понять: **пользователи пришли → вернулись ли через 1 день, 3 дня, 7 дней?**

---

#### Когда применять?

* При оценке **качества продукта** и **пользовательского опыта**
* Для сравнения **эффективности каналов привлечения**
* Чтобы понять **когда** пользователи отваливаются и **на каком шаге** стоит улучшать воронку

---

#### Основные элементы Retention:

| Понятие                | Описание                                                                 |
| ---------------------- | ------------------------------------------------------------------------ |
| **Когорта**            | Группа пользователей, пришедших в один и тот же день (или неделю, месяц) |
| **День жизни**         | Кол-во дней с момента первого действия пользователя                      |
| **Повторное действие** | Возвращение пользователя в продукт (например, покупка или просмотр)      |

In [23]:
# В SQL формируем таблицу с датами событий


In [24]:
# Retention в Pandas


In [26]:
# Визуализация Retention


**Что видим на графике:**

* В строках — пользователи, пришедшие в один день
* В столбцах — доля из них, вернувшихся на `N`‑й день

**Интерпретация:**

| Паттерн                   | Что значит                                  |
| ------------------------- | ------------------------------------------- |
| Падает на 2-й день        | Продукт не цепляет, не возвращаются         |
| Высокое удержание >7 дней | Продукт вызывает привычку                   |